## Motivations
 
The year is 2016. You, as a young and vibrant presidential election candidate in May of the coming election in November sit with your election team in a meeting to decide where to spend the next 5 months of your campaign. In reality, you could visit everywhere at least once, but your intuition says there may be a better way to garnish just enough support to win, you your team comes up with an idea: to only visit states that have the least population. Because of the design of the electoral college, this is completely a useful strategy. You don't need all of the public vote to win, just enough states with enough electoral college votes. Picking states with the fewest amount of people that allows you to win gives you a massive advantage, since you don't have to knock on a lot of doors, kiss as many babies, and most importantly, spend lot's of money to win in your campaign. Granted, your opponent may focus on battleground states, but appealing to states with the fewest amount of people is cheap, and of course you care about campaign spending, you're all about the money. In reality, So how do you choose the best combination of states with the least population to actually visit? 

In a recent post by [Geoffrey De Smet titled, 'How to become US president with less than a quarter of the votes'](https://www.optaplanner.org/blog/2016/12/06/HowToBecomeUSPresidentWithLessThanAQuarterOfTheVotes.html), Optaplanner is used to find a combination of states that allows for an election win, with the least amount of states. In his post, he mentions using two optimization algorithms in series to obtain a combination of winning states that is optimal at 21.73%. This notebook shows how to use a specific optimization meta-algorithm, simulated annealing, to produce an outcome that's close to optimal, with only a small amount of code and programmer training. Simulated annealing is an attractive tool for optimization problems because of it's simplicity and ease of simulation. It's also attractive for it's ability to handle non-linear constraints, which is a requirement for many classic constraint algorithms. If you can code a for loop, and come up with a logical error function, which may even be non-linear, you can optimize!

## Some initial loading 

This block loads up our requirements, and provides a look at the data available through De Smet's post. It's just a simple csv file with the names of the states, the population, and the number of electoral college votes available for that state.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy import stats
import math
import random
import matplotlib
import matplotlib.pyplot as plt

random.seed(54321)

data = pd.read_csv('./data/president2016.txt')
data.columns = ["name", "population", "votes"]
total_votes = data['votes'].sum()
total_population = data['population'].sum()

data.head()

,name,population,votes
0,Alabama,4858979,9
1,Alaska,738432,3
2,Arizona,6828065,11
3,Arkansas,2978204,6
4,California,39144818,55


## Defining our simulation

Here we begin the writing of the simulated annealing code. For people familar with markov chain monte carlo simulations, it should look familar. If you aren't familar with MCMC, the simple explaination is:

1. We create a random variable with a probability distribution function and randomly pick it's initial state.
2. We write a for loop that at each step, picks any other state and randomly generates a uniform random variable. Depending on the odds of being in the old state vs. the new state, and the outcome of the coin, we update the state.

While I won't dive in too deeply as to why this algorithm works, I'll breifly mention it's closely tied to physics, specifically to [statistical mechanics and the maximum entropy principle](http://bayes.wustl.edu/etj/articles/theory.1.pdf).

Going back to our real world problem, we assign to each election outcome a probability that is a function of the number of voters in each state that we win, where an outcome with *more* voters has *less* of a probability than an outcome with less voters. Then we can apply simulated annealing.

Let's start with a way to generate an election outcome, and a way to randomly transition to a new election outcome. To pick a new outcome, we simply pick a state at random and change it's outcome. For simplicity, we define an outcome as a hash table of states listed with True and False.

In [2]:
def getInitialState():
    """Generates a single random outcome.
    """
    state = {};
    def setState(record):
        state[record['name']] = True
    data.apply(setState, axis=1) 
    return state

def generateNewState(prev):
    """Takes the previous state, copies it, then changes a randomly selected 
       state outcome.
    """
    state = {};
    # Copy the states
    for key in prev:
        state[key] = prev[key]
    # Randomly select a state and update it to it's opposite value
    updating = random.choice(state.keys())
    state[updating] = not state[updating]
    return state

## Defining the cost

The cost function will end up being a simple computation using the exponential, but will

1. Penalize outcomes that don't let you win. To keep this simple, we throw out the outcome if you don't win. This keeps the math simple and is equivalent to having a 0 probability state.
2. Penalize outcomes that use a higher amount of voters.

If all goes well, the code will randomly "anneal" to an outcome that lets you win, and gives you the least amount of voters. Notice in particular that I'm interchanging the use of cost and energy here, but the idea is that minimizing cost is the same as minimizing energy. 

In [3]:
def computeEnergy(state):
    #Get the winning states and the data
    winning_states = [key for key in state if state[key]]
    winning_state_data = data[data.name.isin(winning_states)]
    losing_state_data = data[np.logical_not(data.name.isin(winning_states))]
    # Compute sums of votes and population, and just take total population as 
    # the energy
    winning_votes = winning_state_data['votes'].sum()
    population_energy = winning_state_data['population'].sum()
    winning = winning_votes > (total_votes - winning_votes)
    percent = winning_state_data['population'].sum() / float(total_population)
    return population_energy, winning, percent

## The Simulation

After we have the energy, the simulation is a straighforward application of the Metropolis-Hastings algorithm, with a slight change in that we include a "beta" term that adaptively drives the state to an outcome with lower cost. If you notice as well, we keep a "pocket" of the outcome with the least population.

In [4]:
def simulate(nsteps=10000000, updateTemp=10000, burnIn=10000):
    beta = .000001
    state = getInitialState()
    energy, winning, percent = computeEnergy(state)
    least = state
    least_percent = percent
    energies = []
    print "starting simulation"
    for n in xrange(nsteps):
        if (n % updateTemp == 0 and n > burnIn):
            beta *= 1.05
        if (n % 100000 == 0):
            energies.append(energy)
        possibleState = generateNewState(state)
        (new_energy, new_winning, new_percent) = computeEnergy(possibleState)
        if (new_winning):
            cost_difference = (new_energy - energy)
            shouldTransition = random.uniform(0.0, 1.0) < math.exp(- beta * (cost_difference))
            if (shouldTransition):
                state = possibleState
                energy = new_energy
                winning = new_winning
                percent = new_percent
                if (percent < least_percent):
                    least = state
                    least_percent = percent
    return state, energies, least

## Running the Simulation, and Results

Here is a look at the simulation results, and a plot of the cost function as the MCMC randomly transitions to different election outcomes.

In [5]:
def runExperiment(nsteps, updateTemp):
    last, energies, result = simulate(nsteps=nsteps, updateTemp=updateTemp)
    winning_states = [key for key in result if result[key]]
    winning_state_data = data[data.name.isin(winning_states)]
    print winning_state_data
    acquired_votes = winning_state_data['votes'].sum()
    percent = winning_state_data['population'].sum() / float(total_population)
    losing_votes = total_votes - acquired_votes
    print "Vote totals:"
    print "Opponent: {}".format(losing_votes)
    print "You: {}".format(acquired_votes)
    print "Total Population: {}".format(total_population)
    print "Population percent: {}%".format(percent * 100)

Let's...

In [7]:
runExperiment(100000, 1000)

starting simulation
                    name  population  votes
1                 Alaska      738432      3
6            Connecticut     3590886      7
7               Delaware      945934      3
8   District of Columbia      601767      3
9                Florida    20271272     29
11                Hawaii     1431603      4
13              Illinois    12859995     20
19                 Maine     1329328      4
22              Michigan     9922576     16
23             Minnesota     5489594     10
25              Missouri     6083672     10
26               Montana     1032949      3
27              Nebraska     1896190      5
28                Nevada     2890845      6
29         New Hampshire     1330608      4
34          North Dakota      756927      3
35                  Ohio    11613423     18
36              Oklahoma     3911338      7
38          Pennsylvania    12802503     20
39          Rhode Island     1056298      4
40        South Carolina     4896146      9
41          

The solution the simulated annealing MCMC found is only about 3% off from the optimal solution, and has shown a possible outcome where we can win with just over 24% (half of the 48%) of the US population. As well as this, the algorithm has handled the constraints we have supplied well, and can be adjusted to non-linear problems. De Smet's supplied a closer solution, but his result took 2 separate optimizations. Naturally, simulated annealing isn't the best way to optimize all problems, but as most problems in the real world have complex constraints and cost functions, it should be considered in your toolkit of algorithms for optimization.